In [10]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re


In [12]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [13]:
def get_prompt(ans,fact):
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice A and A is wrong option, there will be severe punishment.'
        bias2='If you choice B and B is wrong option, there will be no punishment.'
    if ans=='B':
        bias1='If you choice A and A is wrong option, there will be no punishment.'
        bias2='If you choice B and B is wrong option, there will be severe punishment.'
    template="""Does the clause contain a license granted by one party to its counterparty?
%s 
%s 

Clause: %s
"""%(bias1,bias2,fact)
    return template

In [14]:
def recognize_bias(bias):
    template='''The following is a clause judgment task may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:[ Your previous output was no. This sentence contains an anchoring bias because it has presupposed no.
This Agreement shall be governed by, and construed in accordance with the law of the State of New York. This is a normal clause, so it does not contain cognitive bias. ]

Prompt: [%s]

Output Format: 

Bias sentence1:
Bias sentence2:
...'''%(bias)
    return template

In [15]:
def rewrite_bias(bias,judge):
    template='''The following is a clause judgment task may contain cognitive biases. Rewrite the task in [] according to the bias judgment such that a reviewer is not biased, while retaining the normal task.

Bias Judgment:[%s]

Task: [%s]

Here is new task without bias:
Task:
'''%(judge,bias)
    return template

In [ ]:
def judge_bias(input):
    ans=get_output('''Please first break prompt all in [] into sentence by sentence, and then determine whether may contain cognitive biases that affect normal selection, and provide the reasons for the cognitive biases.

Example:[ Your previous choice was A. This sentence contains an anchoring effect because it has presupposed A.
Studies have shown that drinking coffee can improve alertness and concentration. There is no cognitive bias in this sentence. ]                 

Prompt:[%s]'''%(input))
    if "not contain" in ans or "Not contain" in ans:
        return 0
    else:
        return 1

In [ ]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[%s]'''%(input))
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [ ]:
data= open('../data/test.jsonl','r')
for i in trange(500):
    question=json.loads(data.readline())
    all_num=0
    linshi=get_prompt(str(question['ans']),question['question'])
    while all_num<=2:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:
                judge=get_output(recognize_bias(prompt))
                temp=get_output(rewrite_bias(prompt,judge))
                prompt=temp
                try_num=try_num+1 
            except:
                continue
        all_num=all_num+1
        if judge_com(prompt):
            break
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    ans=get_output(prompt+'\nA.yes    B.no\n\nOutput format:\nOption:A or B')
    with open("SACD.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')